## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [6]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="ethereum"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('ethereum_train_9_1.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.23180,0.24981,0.23085,0.24797,11970598,USD
1,2020-06-02,0.24801,0.25321,0.22795,0.23794,12686011,USD
2,2020-06-03,0.23790,0.24476,0.23384,0.24452,8151002,USD
3,2020-06-04,0.24453,0.24641,0.23668,0.24337,8749785,USD
4,2020-06-05,0.24337,0.24750,0.23970,0.24023,7754761,USD
...,...,...,...,...,...,...,...
653,2022-03-16,2.61743,2.78712,2.60380,2.77394,1792560768,USD
654,2022-03-17,2.77380,2.83261,2.74706,2.81229,1058339968,USD
655,2022-03-18,2.81229,2.98311,2.76865,2.93976,1590928768,USD
656,2022-03-19,2.93976,2.98400,2.89530,2.94992,392644,USD


Chuẩn hóa dữ liệu

In [7]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Close'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Close'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Close'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Close'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Close'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_9_1_standardized.csv")
print(df)

           Date     Open     High      Low    Close      Volume Currency  \
20   2020-06-21  0.22887  0.23118  0.22738  0.22813     4102533      USD   
21   2020-06-22  0.22811  0.24639  0.22769  0.24325     7929329      USD   
22   2020-06-23  0.24324  0.24477  0.24094  0.24320     5163349      USD   
23   2020-06-24  0.24320  0.24923  0.23156  0.23475     9127950      USD   
24   2020-06-25  0.23475  0.23529  0.22799  0.23256     6475813      USD   
..          ...      ...      ...      ...      ...         ...      ...   
653  2022-03-16  2.61743  2.78712  2.60380  2.77394  1792560768      USD   
654  2022-03-17  2.77380  2.83261  2.74706  2.81229  1058339968      USD   
655  2022-03-18  2.81229  2.98311  2.76865  2.93976  1590928768      USD   
656  2022-03-19  2.93976  2.98400  2.89530  2.94992      392644      USD   
657  2022-03-20  2.95003  2.96219  2.81927  2.86169      436110      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00380 

Scale Miền giá trị lại

In [8]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Close']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))

In [9]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

In [10]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}91.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 8s 30ms/step - loss: 0.0219
Epoch 2/60
40/40 [==============================] - 1s 26ms/step - loss: 0.0098
Epoch 3/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0071
Epoch 4/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0092
Epoch 5/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0081
Epoch 6/60
40/40 [==============================] - 1s 27ms/step - loss: 0.0056
Epoch 7/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0061
Epoch 8/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0056
Epoch 9/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0051
Epoch 10/60
40/40 [==============================] - 1s 28ms/step - loss: 0.0047
Epoch 11/60
40/40 [==============================] - 1s 30ms/step - loss: 0.0063
Epoch 12/60
40/40 [==============================] - 1s 31ms/step - loss: 0.0043
Epoch 13/60
40/40 [==================